## **1. Przygotowanie zbioru danych**

### **Uwaga!**
Ze względu na poufność danych, surowe raporty i adnotacje ekspertów nie są zawarte w tym repozytorium.

### **Model bazowy:** speakleash/Bielik-1.5B-v3 (https://huggingface.co/speakleash/Bielik-1.5B-v3)

#### **Import Bibliotek**

In [ ]:
import os
import numpy as np
import pandas as pd
import plotly.express as px
from skmultilearn.model_selection import iterative_train_test_split

In [ ]:
with open('data/data.jsonl', 'r') as f:
    df = pd.read_json(f, lines=True)

#### **1.1. Izolacja rzadkich klas.**

Transformacja list (zmienny typ danych) do krotek (niezmienny typ danych), na potrzeby późniejszych operacji.

In [ ]:
df.labels.transform(tuple)

Odcięcie raportów, których klasa występuje tylko raz, na potrzeby stratyfikacji

In [ ]:
# Ustalamy próg minimalnej liczby wystąpień dla stratyfikacji.
# Próg 3 jest bezpieczny dla 3 podziałów (train/val/test).

MIN_COUNT_FOR_STRATIFICATION = 3

labels_counts = df.labels.value_counts()
rare_labels = labels_counts[labels_counts < MIN_COUNT_FOR_STRATIFICATION].index

# Podział na DataFrame, który możemy stratyfikować, i ten z rzadkimi klasami
df_rare = df[df['labels'].isin(rare_labels)].copy()
df_main = df[~df['labels'].isin(rare_labels)].copy()

print(f"Liczba próbek do stratyfikacji: {len(df_main)}")
print(f"Liczba rzadkich próbek (trafią do zbioru treningowego): {len(df_rare)}")

#### **1.2. Stratyfikacja iteracyjna na głównym zbiorze.**

In [ ]:
# Przygotowanie danych dla scikit-multilearn
# Potrzebujemy tekstu jako macierzy X i etykiet jako macierzy binarnej Y

X_main = df_main['text'].values.reshape(-1, 1)
y_main = np.array(df_main['labels'].tolist())

In [ ]:
# Pierwszy podział: na zbiór treningowy (70%) i tymczasowy testowy (30%)
X_train_main, y_train_main, X_temp, y_temp = iterative_train_test_split(
    X_main, y_main, test_size = 0.3
)

# Drugi podział: dzielimy zbiór tymczasowy (30%) na pół
X_val, y_val, X_test, y_test = iterative_train_test_split(
    X_temp, y_temp, test_size = 0.5
)

#### **1.3. Stworzenie DataFrame dla zbioru treningowego, testowego i walidacyjnego.**

In [ ]:
train_stratified_df = pd.DataFrame({
    'text': X_train_main.ravel(),
    'labels': [list(label) for label in y_train_main]
})

val_df = pd.DataFrame({
    'text': X_val.ravel(),
    'labels': [list(label) for label in y_val]
})

test_df = pd.DataFrame({
    'text': X_test.ravel(),
    'labels': [list(label) for label in y_test]
})

# Dodajemy rzadkie próbki do zbioru treningowego
train_df = pd.concat([train_stratified_df, df_rare], ignore_index=True)

In [ ]:
print(f"Zbiór treningowy: {len(train_df)} raportów")
print(f"Zbiór walidacyjny: {len(val_df)} raportów")
print(f"Zbiór testowy:    {len(test_df)} raportów")

#### **1.4. Wizualizacja podzielonych zbiorów.**

In [ ]:
datasets = {
    "Oryginalny": y_main,
    "Treningowy": y_train_main,
    "Walidacyjny": y_val,
    "Testowy": y_test
}

plot_data = []
kryteria_names = [f"Kryterium {i+1}" for i in range(y_main.shape[1])]

for name, labels_array in datasets.items():
    # Obliczamy dystrybucję (odsetek '1') dla każdej etykiety
    distribution = np.sum(labels_array, axis=0) / len(labels_array)
    for i, percentage in enumerate(distribution):
        plot_data.append({
            "Zbiór Danych": name,
            "Kryterium": kryteria_names[i],
            "Odsetek Wystąpień": percentage
        })

dist_df = pd.DataFrame(plot_data)

In [ ]:
fig = px.bar(
    dist_df,
    x="Kryterium",
    y="Odsetek Wystąpień",
    color="Zbiór Danych",
    barmode="group",
    title="<b>Porównanie Dystrybucji Etykiet Pozytywnych w Zbiorach Danych</b>",
    labels={
        "Odsetek Wystąpień": "Odsetek Wystąpień Etykiety '1'",
        "Kryterium": "Numer Kryterium ESG"
    },
    text_auto='.1%',
)

fig.update_layout(
    yaxis_tickformat=".0%",
    legend_title_text='Zbiór Danych',
    font=dict(family="Arial, sans-serif", size=12),
    template="plotly_white",
    uniformtext_minsize=8, 
    uniformtext_mode='hide'
)

fig.update_traces(textposition='outside')

fig.show()

#### **1.5. Zapis podzielonych zbiorów do plików .jsonl.**

In [ ]:
output_dir = "data/data_split"
os.makedirs(output_dir, exist_ok=True)

datasets_to_save = {
    "train": train_df,
    "validation": val_df,
    "test": test_df
}

for name, df in datasets_to_save.items():
    file_path = os.path.join(output_dir, f"{name}.jsonl")
    df.to_json(
        file_path,
        orient='records',
        lines=True,
        force_ascii=False,
        index=False
    )